In [40]:
import sympy as sm
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
from IPython.display import display, clear_output

#Make paraters global
global c
global mu
global nu
global alpha
global beta
global gamma
global delta

c = 0.19
mu = 0.03
nu = 0.003
alpha = 800
beta = 1.5
gamma = 0.004
delta = 2.2

In [10]:
def xdot(state: tuple , t : np.ndarray, r: float):
    x, y = state
    
    cross = (alpha*x*y) * (1/(beta + x))
    
    dxdt = x*(r - c*x)*(x - mu)* (1/(nu + x)) - cross
    dydt = gamma * cross - delta * y
    return [dxdt, dydt]

In [32]:
def compute_Jacob(F, x_eval, y_eval, r_eval):
    """
    Helper function to compute the Jacobian of the limit cycle
    at the given point.
    """
    F_eval = F.subs([(x, x_eval), (y, y_eval), (r, r_eval)])
    temp = sm.matrix2numpy(F_eval, dtype='float64')
    trace = temp[0, 0] + temp[1, 1]
    return trace

In [ ]:
# Number of R we want to evaluate
num_r = 100
rspan = np.linspace(1.4, 2.6, num_r)

#For limit cycle
timespan = np.linspace(0, 100, num_r)
x0, y0 = 3, 0.010
init = [x0, y0]



period_limits = []
for i, r in enumerate(rspan):
    sol = integrate.odeint(xdot, init, timespan, args=(r, ))
    x, y = sol.T
    xmax = x / max(x)
    
    #finding period ->Taking derivative and plotting zeros
    
    x_max_pts = []
    
    prev_diff = 0
    for j, diff in enumerate(np.diff(xmax)):    
        if prev_diff >= 0 and diff <= 0:
            x_max_pts.append((j, xmax[j]))
        prev_diff = diff
    
    period_limits.append(np.diff(np.array(x_max_pts)[:, 0])[-1])

#Setting up system in sympy
x, y = sm.symbols('x, y', real = True)
r = sm.symbols('r', real = True)
cross = (alpha*x*y) * (1/(beta + x))
dxdt = x*(r - c*x)*(x - mu)* (1/(nu + x)) - cross
dydt = gamma * cross - delta * y
    
#Arrange them into a matrix
F = sm.Matrix([dxdt, dydt])
F_jac = F.jacobian([x, y])
res = []

#Evaluating Jacobian on last period 
for i, r_val in enumerate(rspan):
    integrand = np.zeros(len(rspan))
    integrand[:] = np.nan
    sol = integrate.odeint(xdot, init, timespan, args=(r_val, ))
    x_sol, y_sol = sol.T
    for j in range(len(rspan)):
        integrand[j] = compute_Jacob(F_jac, x_sol[j], y_sol[j], rspan[j])
    
    #Take last period of integrands
    integrand = integrand[-int(period_limits[i]):-1]
    
    #This is pretty much a summation at this point
    
    res.append(np.sum(integrand))
    #Clearing output
    clear_output(wait=True)
    
    print("Percentage done: {:.2f}%".format(100*i/len(rspan)))

In [ ]:
#Plotting results
fig, axs = plt.subplots(1, 1, figsize = (10, 8))
axs.set_title("Evaluated Floquet Multiplier")
axs.set_xlabel("r - value")
axs.set_ylabel("Value for Floquet Multiplier")
axs.grid(True)
axs.set_facecolor("#e1e2e3") 
axs.plot(rspan, res, c="#A23BEC")